<a href="https://colab.research.google.com/github/hadeelfarash/INTERMEDIATE_MACHINE_LEARNING/blob/main/PCA_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.impute  import SimpleImputer

pd.set_option("display.max_columns",200)
pd.set_option("display.max_info_rows",800)
pd.set_option("display.max_info_columns",800)

from sklearn import set_config
set_config(transform_output= "pandas")

In [ ]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def classification_metrics(y_true, y_pred, label='',
                           output_dict=False, figsize=(8,4),
                           normalize='true', cmap='Blues',
                           colorbar=False):
    # Get the classification report
    report = classification_report(y_true, y_pred)
    ## Print header and report
    header = "-"*70
    print(header, f" Classification Metrics: {label}", header, sep='\n')
    print(report)
    ## CONFUSION MATRICES SUBPLOTS
    fig, axes = plt.subplots(ncols=2, figsize=figsize)
    # create a confusion matrix  of raw counts
    ConfusionMatrixDisplay.from_predictions(y_true, y_pred,
                normalize=None, cmap='gist_gray', colorbar=colorbar,
                ax = axes[0],);
    axes[0].set_title("Raw Counts")
    # create a confusion matrix with the test data
    ConfusionMatrixDisplay.from_predictions(y_true, y_pred,
                normalize=normalize, cmap=cmap, colorbar=colorbar,
                ax = axes[1]);
    axes[1].set_title("Normalized Confusion Matrix")
    # Adjust layout and show figure
    fig.tight_layout()
    plt.show()
    # Return dictionary of classification_report
    if output_dict==True:
        report_dict = classification_report(y_true, y_pred, output_dict=True)
        return report_dict


def evaluate_classification(model, X_train, y_train, X_test, y_test,
                         figsize=(6,4), normalize='true', output_dict = False,
                            cmap_train='Blues', cmap_test="Reds",colorbar=False):
    # Get predictions for training data
    y_train_pred = model.predict(X_train)
    # Call the helper function to obtain regression metrics for training data
    results_train = classification_metrics(y_train, y_train_pred, #verbose = verbose,
                                     output_dict=True, figsize=figsize,
                                         colorbar=colorbar, cmap=cmap_train,
                                     label='Training Data')
    print()
    # Get predictions for test data
    y_test_pred = model.predict(X_test)
    # Call the helper function to obtain regression metrics for test data
    results_test = classification_metrics(y_test, y_test_pred, #verbose = verbose,
                                  output_dict=True,figsize=figsize,
                                         colorbar=colorbar, cmap=cmap_test,
                                    label='Test Data' )
    if output_dict == True:
        # Store results in a dataframe if ouput_frame is True
        results_dict = {'train':results_train,
                    'test': results_test}
        return results_dict

## Load the Data

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/CodingDojo/05-IntermediateML/Week17/Data/NHANES_Diet_Diabetes_Risk.xls.csv")
df.head()

SEQN       WTDRD1       WTDR2D  DR1DRSTZ  DR1EXMER  DRABF  DRDINT  \
0  73557  16888.32786  12930.89065         1      49.0    2.0     2.0   
1  73558  17932.14387  12684.14887         1      59.0    2.0     2.0   
2  73559  59641.81293  39394.23671         1      49.0    2.0     2.0   
3  73561  59052.35703  39004.89299         1      63.0    2.0     2.0   
4  73562  49890.82866      0.00000         1      49.0    2.0     1.0   

   DR1DBIH  DR1DAY  DR1LANG  DR1MNRSP  DR1HELPD  DBQ095Z  DBD100  DRQSPREP  \
0      6.0     2.0      1.0       1.0      13.0      3.0     2.0       4.0   
1      4.0     1.0      1.0       1.0      13.0      1.0     2.0       3.0   
2     18.0     6.0      1.0       1.0      13.0      1.0     1.0       2.0   
3     18.0     1.0      1.0       1.0      13.0      4.0     NaN       4.0   
4     11.0     3.0      1.0       1.0      13.0      1.0     3.0       3.0   

   DR1STY  DR1SKY  DRQSDIET  DRQSDT1  DRQSDT2  DRQSDT3  DRQSDT4  DRQSDT5  \
0     2.0     NaN       2.0      NaN      NaN      NaN      NaN      NaN   
1     1.0     1.0       2.0      NaN      NaN      NaN      NaN      NaN   
2     2.0     NaN       1.0      NaN      NaN      NaN      NaN      NaN   
3     2.0     NaN       1.0      NaN      NaN      NaN      NaN      NaN   
4     1.0     1.0       2.0      NaN      NaN      NaN      NaN      NaN   

   DRQSDT6  DRQSDT7  DRQSDT8  DRQSDT9  DRQSDT10  DRQSDT11  DRQSDT12  DRQSDT91  \
0      NaN      NaN      NaN      NaN       NaN       NaN       NaN       NaN   
1      NaN      NaN      NaN      NaN       NaN       NaN       NaN       NaN   
2      NaN      7.0      NaN      NaN       NaN       NaN       NaN       NaN   
3      NaN      NaN      NaN      NaN       NaN       NaN       NaN      91.0   
4      NaN      NaN      NaN      NaN       NaN       NaN       NaN       NaN   

   DR1TNUMF  DR1TKCAL  DR1TPROT  DR1TCARB  DR1TSUGR  DR1TFIBE  DR1TTFAT  \
0      11.0    1574.0     43.63    239.59    176.47      10.8     52.81   
1       8.0    5062.0    338.13    423.78     44.99      16.7    124.29   
2      27.0    1743.0     64.61    224.39    102.90       9.9     65.97   
3       7.0    1421.0     55.24    178.20     87.78      12.3     55.36   
4      17.0    1785.0     55.11    189.59     81.75      22.6     93.92   

   DR1TSFAT  DR1TMFAT  DR1TPFAT  DR1TCHOL  DR1TATOC  DR1TATOA  DR1TRET  \
0    17.819    18.493     8.829     209.0      3.92       0.0     70.0   
1    53.408    35.481    20.505    2584.0     20.94       0.0   1477.0   
2    25.263    20.902    12.953      88.0      8.87       0.0    616.0   
3     4.479    26.216     1.263      41.0     53.17       0.0    738.0   
4    22.155    40.013    23.550     534.0     12.30       0.0    272.0   

   DR1TVARA  DR1TACAR  DR1TBCAR  DR1TCRYP  DR1TLYCO  DR1TLZ  DR1TVB1  DR1TVB2  \
0     110.0      60.0     304.0     278.0    1398.0   430.0    1.057    1.216   
1    1488.0       0.0     145.0      12.0       0.0   899.0    4.178    5.168   
2     655.0      21.0     449.0      35.0     515.0   300.0    1.648    1.747   
3     863.0       0.0    1476.0       0.0       0.0     0.0    1.484    1.720   
4     355.0     172.0     776.0     300.0    2339.0  2101.0    1.227    1.920   

   DR1TNIAC  DR1TVB6  DR1TFOLA  DR1TFA  DR1TFF  DR1TFDFE  DR1TCHL  DR1TVB12  \
0    11.804    0.951     285.0    66.0   219.0     330.0    216.3      2.79   
1    65.396    4.415    1243.0   601.0   641.0    1662.0   2149.0     21.45   
2    18.342    1.906     423.0   290.0   133.0     625.0    161.2      3.78   
3    15.857    2.757     390.0   389.0     1.0     667.0    218.1      8.30   
4    17.119    1.892     458.0    76.0   384.0     511.0    465.7      1.68   

   DR1TB12A  DR1TVC  DR1TVD  DR1TVK  DR1TCALC  DR1TPHOS  DR1TMAGN  DR1TIRON  \
0      0.00   241.4     3.3    29.2     949.0     756.0     206.0      8.41   
1      0.00     2.3    15.2    23.4    3193.0    6427.0     903.0     26.88   
2      1.42   195.1     4.0    40.4     877.0    1198.0  

In [ ]:
# df.drop(columns=['DRQSDT5'], inplace=True)


## Preprocess the Data
Set SEQN as the index
//Train Test Split
//Impute Missing Values with the median value
//Scale

In [ ]:
df = df.set_index("SEQN")
df

WTDRD1        WTDR2D  DR1DRSTZ  DR1EXMER  DRABF  DRDINT  DR1DBIH  \
SEQN                                                                           
73557  16888.32786  12930.890650         1      49.0    2.0     2.0      6.0   
73558  17932.14387  12684.148870         1      59.0    2.0     2.0      4.0   
73559  59641.81293  39394.236710         1      49.0    2.0     2.0     18.0   
73561  59052.35703  39004.892990         1      63.0    2.0     2.0     18.0   
73562  49890.82866      0.000000         1      49.0    2.0     1.0     11.0   
...            ...           ...       ...       ...    ...     ...      ...   
83721  50162.97146  35482.349600         1      49.0    2.0     2.0      6.0   
83723  14116.87893   9661.836186         1      49.0    2.0     2.0     21.0   
83724  32115.84947  46169.498760         1      54.0    2.0     2.0     10.0   
83727  14146.78133  10520.302070         1      59.0    2.0     2.0      8.0   
83729  15272.50520  10401.781170         1      59.0    2.0     2.0      1.0   

       DR1DAY  DR1LANG  DR1MNRSP  DR1HELPD  DBQ095Z  DBD100  DRQSPREP  DR1STY  \
SEQN                                                                            
73557     2.0      1.0       1.0      13.0      3.0     2.0       4.0     2.0   
73558     1.0      1.0       1.0      13.0      1.0     2.0       3.0     1.0   
73559     6.0      1.0       1.0      13.0      1.0     1.0       2.0     2.0   
73561     1.0      1.0       1.0      13.0      4.0     NaN       4.0     2.0   
73562     3.0      1.0       1.0      13.0      1.0     3.0       3.0     1.0   
...       ...      ...       ...       ...      ...     ...       ...     ...   
83721     1.0      1.0       1.0      13.0      1.0     1.0       2.0     2.0   
83723     7.0      1.0       1.0      13.0      4.0     NaN       4.0     2.0   
83724     3.0      1.0       1.0      13.0      4.0     NaN       1.0     2.0   
83727     7.0      1.0       1.0      13.0      1.0     1.0       3.0     2.0   
83729     6.0      1.0       1.0      13.0      4.0     NaN       3.0     2.0   

       DR1SKY  DRQSDIET  DRQSDT1  DRQSDT2  DRQSDT3  DRQSDT4  DRQSDT5  DRQSDT6  \
SEQN                                                                            
73557     NaN       2.0      NaN      NaN      NaN      NaN      NaN      NaN   
73558     1.0       2.0      NaN      NaN      NaN      NaN      NaN      NaN   
73559     NaN       1.0      NaN      NaN      NaN      NaN      NaN      NaN   
73561     NaN       1.0      NaN      NaN      NaN      NaN      NaN      NaN   
73562     1.0       2.0      NaN      NaN      NaN      NaN      NaN      NaN   
...       ...       ...      ...      ...      ...      ...      ...      ...   
83721     NaN       2.0      NaN      NaN      NaN      NaN      NaN      NaN   
83723     NaN       2.0      NaN      NaN      NaN      NaN      NaN      NaN   
83724     NaN       2.0      NaN      NaN      NaN      NaN      NaN      NaN   
83727     NaN       2.0      NaN      NaN      NaN      NaN      NaN      NaN   
83729     NaN       1.0      1.0      NaN      NaN      NaN      NaN      NaN   

       DRQSDT7  DRQSDT8  DRQSDT9  DRQSDT10  DRQSDT11  DRQSDT12  DRQSDT91  \
SEQN                                                                       
73557      NaN      NaN      NaN       NaN       NaN       NaN       NaN   
73558      NaN      NaN      NaN       NaN       NaN       NaN       NaN   
73559      7.0      NaN      NaN       NaN       NaN       NaN       NaN   
73561      NaN      NaN      NaN       NaN       NaN       NaN      91.0   
73562      NaN      NaN      NaN       NaN       NaN       NaN       NaN   
...        ...      ...      ...       ...       ...       ...       ...   
83721      NaN      NaN      NaN       NaN       NaN       NaN       NaN   
83723      NaN      NaN      NaN       NaN       NaN       NaN       NaN   
83724      NaN      NaN      NaN       NaN       NaN       NaN       NaN   
83727      NaN      NaN    

In [ ]:
target = 'Diabetes_Risk'

In [ ]:
y = df[target]
X = df.drop(columns = [target])

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=321)

## Impute Missing Values with the median value


In [ ]:
df.isna().sum()

WTDRD1              0
WTDR2D            559
DR1DRSTZ            0
DR1EXMER          518
DRABF             559
                 ... 
DRD370TQ         5664
DRD370U          2665
DRD370UQ         6373
DRD370V          2666
Diabetes_Risk       0
Length: 168, dtype: int64

## Scale

In [ ]:
# Instantiate Standard Scaler
scaler = StandardScaler()
# Fit & transform data.
X_train_tf = scaler.fit_transform(X_train)
X_test_tf = scaler.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


## Check for Class Balance


In [ ]:
# Check for class balance of target
y_train.value_counts(1)

0    0.824167
2    0.089522
1    0.086311
Name: Diabetes_Risk, dtype: float64

In [ ]:
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE,SMOTENC

# Assuming X_train_tf contains NaN values
imputer = SimpleImputer(strategy='median')  # Choose appropriate strategy
X_train_imputed = imputer.fit_transform(X_train_tf)
X_test_imputed = imputer.fit_transform(X_test_tf)

smote = SMOTE()
# Apply SMOTE on the imputed data
X_train_sm, y_train_sm = smote.fit_resample(X_train_imputed, y_train)



# from imblearn.over_sampling import SMOTE,SMOTENC
# smote = SMOTE()
# X_train_sm, y_train_sm = smote.fit_resample(X_train_tf, y_train)
y_train_sm.value_counts()

0    4106
1    4106
2    4106
Name: Diabetes_Risk, dtype: int64

## Model - No PCA

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import datetime as dt

# Record the start time
start = dt.datetime.now()

knn = KNeighborsClassifier()  # Initialize KNN classifier with default parameters

knn.fit(X_train_sm, y_train_sm)  # Fit the KNN model

# Record the end time and calculate duration
end = dt.datetime.now()
dur_baseline = end - start

evaluate_classification(knn, X_train_sm,y_train_sm, X_test_imputed, y_test)
print(f'Training time was: {dur_baseline}')


----------------------------------------------------------------------
 Classification Metrics: Training Data
----------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.75      0.56      0.64      4106
           1       0.80      0.92      0.86      4106
           2       0.81      0.89      0.85      4106

    accuracy                           0.79     12318
   macro avg       0.78      0.79      0.78     12318
weighted avg       0.78      0.79      0.78     12318



ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- DRQSDT12


## Transform the data with PCA​

In [ ]:
pca = PCA(n_components=3)
X_train_pca = pca.fit_transform(X_train_sm)
X_test_pca = pca.transform(X_test_imputed)
X_train_pca.head()

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- DRQSDT5


In [ ]:
# Record the start time
start = dt.datetime.now()

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_pca, y_train_sm)

# Record the end time and calc duration
end = dt.datetime.now()
dur_pca = end-start

evaluate_classification(clf, X_train_pca,y_train_sm, X_test_pca, y_test)
print(f'Training time was: {dur_pca}')

NameError: name 'X_test_pca' is not defined

##   Compare Models

In [ ]:
# Compare speeds before and after PCA
compare_speed = dur_baseline/dur_pca
print(f"Using PCs was {compare_speed:.2f} times faster!")

*Q1. How many features were used in the first model, and how many PCs were used in the PCA model?*

*Q2. Which model performed the best on the test set?*

*Q3. Which model was the fastest at making predictions?*